This script loads behavioral mice data (from `biasedChoiceWorld` sessions and, separately, the last three sessions of training) only from mice that pass a given (stricter) training criterion defined in a *schema* uploaded by Anne Urai.
The data are slightly reformatted and saved as `.csv` files.

In [23]:
import datajoint as dj
dj.config['database.host'] = 'datajoint.internationalbrainlab.org'

from ibl_pipeline import subject, acquisition, action, behavior, reference, data
from ibl_pipeline.analyses.behavior import PsychResults
from ibl_pipeline.utils import psychofit as psy
from ibl_pipeline.analyses import behavior as behavioral_analyses

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [28]:
import os
myPath = r"C:\Users\Luigi\Documents\GitHub\ibl-changepoint\data\tics2020" # Write here your data path
os.chdir(myPath)

In [29]:
# Get example mice
mice_names = ['CSHL_007','KS014']

In [30]:
sess_train  = (acquisition.Session & 'task_protocol LIKE "%training%"')
sess_stable = (acquisition.Session & 'task_protocol LIKE "%biased%"')
stable_mice_names = list()

# Perform at least this number of sessions
MinSessionNumber = 1 

def get_mouse_data(df):
    position_deg = 1    # Stimuli appear at +/- 35 degrees

    # Create new dataframe
    datamat = pd.DataFrame()
    datamat['trial_num'] = df['trial_id']
    datamat['session_num'] = np.cumsum(df['trial_id'] == 1)
    datamat['stim_probability_left'] = df['trial_stim_prob_left']
    signed_contrast = df['trial_stim_contrast_right'] - df['trial_stim_contrast_left']
    datamat['contrast'] = np.abs(signed_contrast)*100
    datamat['position'] = np.sign(signed_contrast)*position_deg
    datamat['response_choice'] = df['trial_response_choice']
    datamat.loc[df['trial_response_choice'] == 'CCW','response_choice'] = 1
    datamat.loc[df['trial_response_choice'] == 'CW','response_choice'] = -1
    datamat.loc[df['trial_response_choice'] == 'No Go','response_choice'] = 0
    datamat['trial_correct'] = np.double(df['trial_feedback_type']==1)
    datamat['reaction_time'] = df['trial_response_time'] - df['trial_stim_on_time'] # double-check

    # Since some trials have zero contrast, need to compute the alleged position separately
    datamat.loc[(datamat['trial_correct'] == 1) & (signed_contrast == 0),'position'] = \
        datamat.loc[(datamat['trial_correct'] == 1) & (signed_contrast == 0),'response_choice']*position_deg
    datamat.loc[(datamat['trial_correct'] == 0) & (signed_contrast == 0),'position'] = \
        datamat.loc[(datamat['trial_correct'] == 0) & (signed_contrast == 0),'response_choice']*(-position_deg)
    
    # Remove trials with unknown position or response
    datamat.drop(datamat[datamat['position'] == 0].index, inplace=True)
    datamat.drop(datamat[datamat['response_choice'] == 0].index, inplace=True)
    
    return datamat

In [31]:
# Loop over all mice
for mouse_nickname in mice_names:
    
    mouse_subject = {'subject_nickname': mouse_nickname}
    
    # Get mouse data for biased sessions
    behavior_stable = (behavior.TrialSet.Trial & (subject.Subject & mouse_subject)) \
        * sess_stable.proj('session_uuid','task_protocol') * subject.Subject.proj('subject_nickname') \
        * subject.SubjectLab.proj('lab_name')
    df = pd.DataFrame(behavior_stable.fetch(order_by='subject_nickname, session_start_time, trial_id'))

    if len(df) > 0:           # The mouse has performed in at least one stable session with biased blocks            
        datamat = get_mouse_data(df)

        # Take mice that have performed a minimum number of sessions
        if np.max(datamat['session_num']) >= MinSessionNumber:

            # Save dataframe to CSV file
            filename = mouse_nickname + '_biased.csv'
            datamat.to_csv(filename,index=False)
            stable_mice_names.append(mouse_nickname)

            # Get mouse last sessions of training data
            behavior_train = (behavior.TrialSet.Trial & (subject.Subject & mouse_subject)) \
                * sess_train.proj('session_uuid','task_protocol') * subject.Subject.proj('subject_nickname') \
                * subject.SubjectLab.proj('lab_name')
            df_train = pd.DataFrame(behavior_train.fetch(order_by='subject_nickname, session_start_time, trial_id'))

            datamat_train = get_mouse_data(df_train)
            # Nlast = np.max(datamat_train['session_num']) - 3
            Nlast = 0
            datamat_final = datamat_train[datamat_train['session_num'] > Nlast]

            # Save final training dataframe to CSV file
            filename = mouse_nickname + '_train.csv'
            datamat_final.to_csv(filename,index=False)
            

print(stable_mice_names)

['CSHL_007', 'KS014']


In [17]:
datamat

,trial_num,session_num,stim_probability_left,contrast,position,response_choice,trial_correct,reaction_time
0,1,1,0.8,0.2500,-1.0,-1,1.0,0.667800
1,2,1,0.8,0.1250,1.0,-1,0.0,6.079000
2,3,1,0.8,0.1250,-1.0,1,0.0,2.720100
3,4,1,0.8,0.2500,-1.0,-1,1.0,0.756300
4,5,1,0.8,1.0000,-1.0,-1,1.0,0.783100
...,...,...,...,...,...,...,...,...
26982,249,68,0.2,1.0000,1.0,0,0.0,59.996201
26983,250,68,0.2,0.2500,1.0,0,0.0,59.997501
26984,251,68,0.2,0.0000,0.0,0,0.0,60.005701
26985,252,68,0.2,0.1250,1.0,1,1.0,11.386901
